In [2]:
import glob
import shutil 
from netCDF4 import Dataset
import numpy as np
import os

from obspy.core import Stream, Trace, UTCDateTime, Stats
from obspy.core.event import read_events


In [6]:
ModelName = 'globalmodel0013'
RunPath = './Runs/%s' %ModelName

NETCDFDir = '/raid1/TomoNet/Runs/Sample/%s' %ModelName
if not os.path.exists(NETCDFDir):
    os.mkdir(NETCDFDir)
StationInfo = np.loadtxt(RunPath+'/input/Synthetic_Stations',dtype=str, skiprows=1)
stalatlon_dict = {}
for item in StationInfo:
    stkey = item[1]+'.'+item[0]
    lat = float(item[2])
    lon = float(item[3])
    if stkey not in stalatlon_dict.keys():
        stalatlon_dict[stkey] = []
    stalatlon_dict[stkey].append((lat, lon))

# from obspy.taup import TauPyModel
# model = TauPyModel(model="prem")
# OUTPUTFormat = 'Default'

StartTime, EndTime = 0, 3000
resample_rate = 1.
lowpass_freq = 1/50

PointPerTrace = int((EndTime - StartTime)/resample_rate)+1

istation = 0

if os.path.exists(NETCDFDir+'/seismogram.nc'):
    os.remove(NETCDFDir+'/seismogram.nc')
nc = Dataset(NETCDFDir+'/seismogram.nc', 'w')
nc.createDimension('npoint', size=PointPerTrace*len(stalatlon_dict))
nc.createVariable('x', float, dimensions=('npoint'))
nc.createVariable('y', float, dimensions=('npoint'))
nc.createVariable('z', float, dimensions=('npoint'))
nc.createVariable('time', float, dimensions=('npoint'))
nc.createVariable('disp_x', float, dimensions=('npoint'))
nc.createVariable('disp_y', float, dimensions=('npoint'))
nc.createVariable('disp_z', float, dimensions=('npoint'))


GSNDir = RunPath + '/output/stations/Synthetic_Stations'

# read rank-station info
rank_station_info = np.loadtxt(GSNDir + '/rank_station.info', dtype=str, skiprows=1)

# dict: mpi-rank -> [station keys]
rank_station_dict = {}
for item in rank_station_info:
    rank = item[0]
    stkey = item[1]
    inrank_index = item[2]
    # initialize with an empty array if rank does not exists in rank_station_dict
    if rank not in rank_station_dict.keys():
        rank_station_dict[rank] = []
    # append the station
    rank_station_dict[rank].append([stkey, inrank_index])
        
# loop over mpi-ranks to read data
for rank in rank_station_dict.keys():
    f = Dataset(GSNDir + '/axisem3d_synthetics.nc.rank%s' %rank, 'r')
    time = f.variables['data_time'][:]

    for [StationName, inrank_index] in rank_station_dict[rank]:

        lat, lon = stalatlon_dict[StationName][0][0], stalatlon_dict[StationName][0][1]
        colat = 90 - lat

        theta = np.radians(colat)
        phi = np.radians(lon)

        stax = 6371e3*np.sin(theta)*np.cos(phi)
        stay = 6371e3*np.sin(theta)*np.sin(phi)
        staz = 6371e3*np.cos(theta)

        # trace header
        stats = Stats()
        stats.starttime = UTCDateTime(time[0])
        stats.delta = UTCDateTime(time[1] - time[0])
        stats.npts = len(time)

        # stream
        stream = Stream()
        for ich, ch in enumerate('spz'):
            stats.channel = ch  
            # default unit is km
            stream.append(Trace(f.variables['data_wave'][int(inrank_index)][ich], header=stats))

        # arrivals = model.get_travel_times(source_depth_in_km=400,
        #                                 distance_in_degree=colat,
        #                                 phase_list=["P", "Pdiff"])
        # arrivals[0].time
        # process (filter, resample, slice, ...)
        stream.filter('lowpass', freq=lowpass_freq)
        stream.resample(resample_rate)
        # stream = stream.slice(UTCDateTime(int(arrivals[0].time)+StartTime), UTCDateTime(int(arrivals[0].time)+EndTime))
        stream = stream.slice(UTCDateTime(StartTime), UTCDateTime(EndTime))
        npoint_persta = len(stream[0].data)
       
        nc['x'][istation*npoint_persta:(istation+1)*npoint_persta] = stax * np.ones(npoint_persta)
        nc['y'][istation*npoint_persta:(istation+1)*npoint_persta] = stay * np.ones(npoint_persta)
        nc['z'][istation*npoint_persta:(istation+1)*npoint_persta] = staz * np.ones(npoint_persta)

        # nc['time'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].times() + int(arrivals[0].time) + StartTime
        nc['time'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].times()

        nc['disp_x'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].data * np.cos(phi) - stream[1].data * np.sin(phi)
        nc['disp_y'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].data * np.sin(phi) + stream[1].data * np.cos(phi)
        nc['disp_z'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[2].data

        istation += 1
# # print & plot
# print(stream)
# stream.plot()
# nc.createVariable('time', float, dimensions=('npoint'))
# nc['time'][:] = stream[0].times()
nc.close()

print(NETCDFDir+'/seismogram.nc', " set up and saved!")

/raid1/TomoNet/Runs/Sample/globalmodel0013/seismogram.nc  set up and saved!


In [5]:
npoint_persta
PointPerTrace

3000

In [21]:
import shutil
# Move netcdf file
# shutil.copy(RunPath+'/input/box_tomography.nc',NETCDFDir)
shutil.copy(RunPath+'/tools/UnifromEarthCore.bm',NETCDFDir)

'/raid1/TomoNet/Runs/Sample/model0011/UnifromEarthCore.bm'